In [1]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import time
import pandas as pd
import gurobipy as grb
import datetime
import random
np.set_printoptions(edgeitems=15,linewidth=600)

%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.optimization.MIP.mip_inputs import *
from src.optimization.MIP.mip_model import *
from src.optimization.MIP.mip_outputs import *

In [2]:
start = time.time()

### Model Inputs 

In [3]:
UI_params = {
    'initial_procurement_budget':1300000,
    'procurement_budget_rate':0.03,
    'initial_operations_budget':1000000,
    'maintenance_budget_rate':0.03,
    'planning_interval':[2022,2037],
    'emissions_baseline': 2031705,#metric tons
    'emissions_target_pct_of_base':1.0,
    'min_miles_replacement_threshold':150000,#miles
    'min_vehicle_age_replacement_threshold':60,#years
    'max_vehicles_per_station':1000,
    'objective_weights':{'cost':0.70,'emissions':0.30},
}

In [4]:
data = pd.read_excel('../data/17MAR_data_template.xlsx').head(50)
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')

In [5]:
inputs = MIP_Inputs(data,UI_params)

In [6]:
outputs = MIP_Outputs(data,UI_params)
print("--- %s seconds ---" % (time.time() - start))

Using license file C:\Users\elynch\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolSolutions to 500
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 83 rows, 3302 columns and 62082 nonzeros
Model fingerprint: 0xc21526d2
Variable types: 2 continuous, 3300 integer (3300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [1e+01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
Found heuristic solution: objective 277286.10000
Presolve removed 17 rows and 0 columns
Presolve time: 0.02s
Presolved: 66 rows, 3302 columns, 9216 nonzeros
Variable types: 2 continuous, 3300 integer (3300 binary)

Root relaxation: objective 2.211474e+05, 140 iterat

In [7]:
optimal_obj,optimal_solution = outputs.get_optimal_solution()

In [8]:
alt_solutions = outputs.get_alternative_solutions()
selected_alternative_solutions = outputs.select_alternative_solutions(alt_solutions,optimal_obj)

In [9]:
options = selected_alternative_solutions

In [10]:
option1 = pd.DataFrame(options[0,'schedules'],columns=['vehicle_idx','solution_idx']+inputs.years)

In [11]:
data = data.merge(option1,on='vehicle_idx',how='left')

In [12]:
data.groupby(['county'])[inputs.years].sum().replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
county,,,,,,,,,,,,,,,,
Baltimore County,20,22,4,,,,,4,7,9,5,1,2,4,4,7


In [13]:
data.groupby(['dept_name'])[inputs.years].sum().replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
dept_name,,,,,,,,,,,,,,,,
Baltimore County Department of Library,1,,,,,,,,,1,,,,,,
Bureau of Corrections,2,1,,,,,,,,1,,,,1,1,
County Sheriff,1,,,,,,,,,,,,,,,
DPW: Bureau of Utilities (Sewer/Water Opns./Maintenance),3,1,,,,,,,,,1,,,,,
DPW: Bureau of Utilties (Pumping/Treatment Plants),4,4,1,,,,,,,2,2,,1,1,,
DPW: Equipment Maintenance,,2,,,,,,,1,,,,,1,,1
DPW: Highways/General Oper/Maintenance,1,,,,,,,1,,,,,,,1,
DPW: Solid Waste Management (Refuse Disposal),2,1,,,,,,2,1,,,,,,2,1
DPW: Traffic Signal Operations,,1,,,,,,,,,1,,,,,


In [14]:
data.groupby(['dept_name','vehicledescription'])[inputs.years].sum().replace({0:''})

2022  \
dept_name                                          vehicledescription                     
Baltimore County Department of Library             2002 FORD VAN CARGO'               1   
Bureau of Corrections                              2001 GMC VAN 12PSG'                    
                                                   2005 FORD CROWN VICTORIA'          1   
                                                   2006 FORD CROWN VICTORIA'          1   
County Sheriff                                     2005 CHEVROLET VAN,PRISONER TP'    1   
DPW: Bureau of Utilities (Sewer/Water Opns./Mai... 2005 CHEVROLET 3/4 TON PU'         3   
DPW: Bureau of Utilties (Pumping/Treatment Plants) 2001 CHEVROLET 1.0 TON UT'             
                                                   2003 CHEVROLET C4500 UTIL BODY'    1   
                                                   2004 FORD F450'                        
                                                   2005 CHEVROLET C4500 UTIL BODY'    3   
DPW: Equipment Maintenance                         2003 CHEVROLET C4500 UTIL BODY'        
                                                   2004 CHEVROLET 3/4 TON PU'             
DPW: Highways/General Oper/Maintenance             2005 CHEVROLET 3/4 TON PU'         1   
DPW: Solid Waste Management (Refuse Disposal)      1999 GMC 3/4 TON PU'               1   
                                                   2003 CHEVROLET 3/4 TON PU'         1   
                                                   2006 JEEP LIBERTY'                     
DPW: Traffic Signal Operations                     2000 GMC 3/4 TON PU'                   
Department of Health (Bioterrorism Program)        2000 GMC 3/4 TON PU'                   
Fire Department - Operations                       2005 CHEVROLET SUBURBAN'               
OIT: Electronic Services/Telecommunications        2000 CHEVROLET VAN CARGO'              
                                                   2000 GMC 1.0 TON UT'                   
                                                   2003 FORD E350'                    1   
Police Department - Administrative/Technical Se... 2002 FORD TAURUS'                      
                                                   2003 CHEVROLET Astro'              1   
Police Department - Office of the Chief            2005 Chrysler Sebring'             1   
Police Department - Operations                     2005 JEEP LIBERTY'                     
                                                   2006 FORD CROWN VICTORIA'          1   
                                                   2006 JEEP LIBERTY'                     
Police Department - Support Operations             2003 Harley Davidson FLHT'             
                                                   2005 CHEVROLET SUBURBAN'               
Property Management - Building Maintenance         2002 FORD VAN CARGO'               1   
                                                   2003 CHEVROLET VAN CARGO'          1   
Recreation and Parks (Opns./Maintnenance)          2004 CHEVROLET 3/4 TON PU'             
Recreation and Parks (Recreation Services)         2001 GMC VAN CARGO'                    
Vehicle Operations and Maintenance                 2005 FORD WRECKER'                     

                                                                                   2023  \
dept_name                                          vehicledescription                     
Baltimore County Department of Library             2002 FORD VAN CARGO'                   
Bureau of Corrections                              2001 GMC VAN 12PSG'                1   
                                                   2005 FORD CROWN VICTORIA'              
                                                   2006 FORD CROWN VICTORIA'              
County Sheriff                                     2005 CHEVROLET VAN,PRISONER TP'        
DPW: Bureau of Utilities (Sewer/Water Opns./Mai... 2005 CHEVROLET 3/4 TON PU' 

In [15]:
data.groupby(['equipmentid'])[inputs.years].sum().replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
equipmentid,,,,,,,,,,,,,,,,
24399.0,1,,,,,,,1,,,,,,,1,
26142.0,1,,,,,,,,,1,,,,,,
26145.0,1,,,,,,,,,,,,,,,
26521.0,,1,,,,,,,,,1,,,,,
26526.0,,1,,,,,,,,,,,,,,
26563.0,,1,,,,,,,1,,,,,,,1
26566.0,,1,,,,,,,,1,,,,,,
26783.0,,1,,,,,,,,,1,,,,,
26785.0,,1,,,,,,,,,,,,,1,


In [16]:
print('   acquisition_costs')
pd.DataFrame(pd.DataFrame(options[0,'acquisition_costs'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   acquisition_costs


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,1298000,1281000,221500,0,0,0,0,163000,375500,534000,436500,31500,211500,212500,173000,375500


In [17]:
print('   consumables_costs')
pd.DataFrame(pd.DataFrame(options[0,'consumables_costs'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   consumables_costs


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,26077,19494,19318,19318,19318,19318,19318,19318,19318,19318,19318,19318,19318,19318,19318,19318


In [18]:
print('   mx_costs')
pd.DataFrame(pd.DataFrame(options[0,'mx_costs'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   mx_costs


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,52600,51600,53200,55000,55600,57400,57800,57400,56400,55800,53000,52200,53800,53600,53000,52200


In [19]:
print('   emissions_amts')
pd.DataFrame(pd.DataFrame(options[0,'emissions_amts'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   emissions_amts


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,16949,2331,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021


In [20]:
print('   EV/Hybrid Inventory')
pd.DataFrame(pd.DataFrame(options[0,'conversions'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   EV/Hybrid Inventory


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,20,42,46,46,46,46,46,46,46,46,46,46,46,46,46,46
